# Retrieval-augmented Generation (RAG) with german doctor ratings  

## Ingesting and retrieving reviews with **llama-index** and **Huggingface**
In this notebook we will be looking at the ingestion and retrieval process of a Retrieval-augmented Generation system.

You will build the ingestion pipeline to read in german doctor reviews. Along the way you will compute text embeddings, store the information into Vector-Database and setup the components to perform the retrieval of relevant information for a user query.

To faciliate the data transformation we will rely on the [**LlamaIndex**](https://docs.llamaindex.ai/) package and make use of [**HuggingFace's Inference API**](https://huggingface.co/docs/api-inference/en/index)

In [ ]:
!pip install llama-index==0.12.24 llama-index-vector-stores-chroma==0.4.1 llama-index-llms-huggingface-api==0.4.1 llama-index-embeddings-huggingface-api==0.3.0

In [ ]:
import re
import tqdm
from datetime import datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import shutil

from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex

from llama_index.vector_stores.chroma import ChromaVectorStore

import chromadb
import nest_asyncio

tqdm.tqdm.pandas()
sns.set_style('whitegrid')

pd.options.display.max_colwidth = 600
pd.options.display.max_rows = 400

## Setting up Huggingface
To be able to access the Huggingface Inference API we will have to use an access token.

You will need your Huggingface account. If you not have done already, perform the sign up on https://huggingface.co

Then create a fresh access token under https://huggingface.co/settings/tokens
Make sure to
- select "Fine-grained" as token type
- give the token a name and
- tick under `Inference`
  - `Make calls to inference providers` and
  - `Make calls to Inference Endpoints`
- Store the generated access token somewhere safely

Finally, execute the cell bellow and copy&paste the token into the provided field to configure the access of this notebook.

In [ ]:
from huggingface_hub import login

login()

## Preparing the **German language reviews of doctors by patients 2019** dataset

In [ ]:
!wget -O reviews.zip https://query.data.world/s/v5xl53bs2rgq476vqy7cg7xx2db55y
!unzip reviews.zip

### Data loading
The dataset contains patients' text comments in german and a coresponding ratings.

We will intermediately load the data into a Pandas DataFrame. Then, to ease the loading process with LlamaIndex, write each review into a separate file.

We limit ourselfs to the first 10 entries of the dataset not to overuse available credits of the Huggingface Inference API.

In [ ]:
# read data from csv
reviews = pd.read_csv("german_doctor_reviews.csv")[:10]

In [ ]:
# create the target folder for the reviews
dataset_path = Path('german_doctor_reviews')

if dataset_path.exists():
    shutil.rmtree(dataset_path)

dataset_path.mkdir(parents=True, exist_ok=True)

# write one file per review
for idx, row in tqdm.tqdm(reviews.iterrows(), total=len(reviews)):
    with (dataset_path / f"review_{idx}.txt").open("w") as fp:
        fp.write(row["comment"])

### Loading the documents
Now that the data is properly stored on the file system we can leverage LlamaIndex' `SimpleDirectoryReader` to skim through the directory and load each document into memory

In [ ]:
reader = SimpleDirectoryReader(input_dir=str(dataset_path))
documents = reader.load_data()
len(documents)

## Ingesting the document into a vector store

### Defining the vector store
We will be using [`chroma` ](https://github.com/chroma-core/chroma) as our embedding or vector database. It natively supports embeddings and vector similarity searches and can very easily be integrated into the common GenAI orchestration frameworks.

In [ ]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection(name="german_doctor_reviews")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

### Defining the embedding model
Experimenting with different embedding models is made very easy by using Huggingface Inference API. Popular models are hosted on their infrastructure.

For our case we will be using [`BAAI/bge-small-en-v1.5`](https://huggingface.co/BAAI/bge-small-en-v1.5) an embedding model trained and published by the Beijing Academy of Artificial Intelligence.

The BGE (BAAI General Embedding) focuses on retrieval-augmented LLMs

In [ ]:
embed_model = HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")

### Defining the pipeline
Now its time to put everything together: LlamaIndex' `IngestionPipeline` allows us to defeine transformation steps and the target vector store where the ingested data should be stored.

Our pipeline will use a `SentenceSplitter` to chunk up the different reviews by sentence. Then the pipeline will pass each chunk to our embedding model to obtain a vector representation of the text.
Lastly, the pipeline will load each chunk along with its embedding into the chroma vector store.

In [ ]:
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        embed_model,
    ],
    vector_store=vector_store,
)


### Indexing the document
We are all set now! Lets execute the pipeline and ingest all our reviews into the vector database

In [ ]:
nodes = await pipeline.arun(documents=documents)
len(nodes)

## Data analysis
The pipeline processed all the documents and returned the nodes that were created along the process.

Lets explore the returned data in more detail

In [ ]:
##########################
## YOUR CODE HERE START ##
##########################

# Have a look at the returned nodes.
# Print the text and its corresponding embedding of the first node

print(....)

##########################
## YOUR CODE HERE END ##
##########################

To get a better feeling what the embeddings represent we can compute the cosine distance between all the embedding vectors.

Cosine distance is a measure of dissimilarity between two vectors in an inner product space. It is derived from the **cosine similarity**, which computes the cosine of the angle between two vectors. T

The cosine distance ranges from **0** (when vectors are identical) to **2** (when vectors point in exactly opposite directions).

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
filenames = [node.metadata["file_name"] for node in nodes]

##########################
## YOUR CODE HERE START ##
##########################

# compute the distances for each embedding to each other embedding

embeddings = [....]
distances = ....

##########################
## YOUR CODE HERE END ##
##########################

plt.imshow(distances, interpolation='nearest', cmap='viridis')
plt.colorbar(label='Cosine Distance')

plt.xticks(ticks=np.arange(len(filenames)), labels=filenames, rotation=90)
plt.yticks(ticks=np.arange(len(filenames)), labels=filenames)
plt.title('Cosine Distance Heatmap between Text Embeddings')

Which reviews are similiar, which are very different?

Can you see a pattern in the data?

## Retrieval-augmented Generation in action
`LlamaIndex` allows us to turn our vector database into an index that we can query with a string.

We will need to define the `VectorStoreIndex` using our previously created `vector_store` and also pass along our `embed_model`. This will allow the index to turn a user query into an embedding an search for the most similar entries.

These entries will then subsequently be used to formulate an answer to the user inquery.

In [ ]:
nest_asyncio.apply()  # This is needed to run the query engine

llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)

In [ ]:
user_query = "Welcher Doktor ist gut in München für Zahnprobleme?"

##########################
## YOUR CODE HERE START ##
##########################

# Use the query_engine to answer the user query

response = ....

##########################
## YOUR CODE HERE END ##
##########################


print('Query:', user_query)
print('Response:', response.response)
print('References:')
for node in response.source_nodes:
    print(f'{node.metadata["file_name"]} -{node.text[:200]}')